Com esta query, construiu-se o dataset de identificação os CNPJs ocultados na base de Operações Indiretas Automáticas.

In [ ]:
import pandas as pd
from google.colab import auth
from google.cloud import bigquery

In [ ]:
project_id = 'testebigquery-273917'
client = bigquery.Client(project=project_id)

In [ ]:
client.query('''
CREATE TEMP FUNCTION token_set_ratio(a STRING, b STRING)
RETURNS FLOAT64
LANGUAGE js AS """
  return fuzzball.token_set_ratio(a, b);
"""
OPTIONS (library="gs://bndes_dadosabertos/js/fuzzball.umd.min.js");

CREATE TABLE bndes.operacoes_indiretas_cnpj AS
SELECT
    t1.cliente as nome_bndes,
    t2.razao_social as nome_receita,
    t1.cpf_cnpj as cnpj_bndes,
    t2.cnpj as cnpj_receita,
    token_set_ratio(t1.cliente, t2.razao_social) as score
FROM (
SELECT DISTINCT
    t1.cliente,
    t1.cpf_cnpj,
    ARRAY_TO_STRING(REGEXP_EXTRACT_ALL(t1.cpf_cnpj, '[0-9]'), '') as cnpj9
FROM `testebigquery-273917.bndes.operacoes_indiretas_automaticas` AS t1
) AS t1
LEFT JOIN `testebigquery-273917.bndes.empresas` AS t2
ON t1.cnpj9=substring(lpad(cast(t2.cnpj as string), 14, "0"), 4, 9);

WITH BASE AS (
SELECT *,
ROW_NUMBER() OVER (PARTITION BY nome_bndes ORDER BY score DESC) AS rank 
FROM `testebigquery-273917.bndes.operacoes_indiretas_cnpj`
)
SELECT * FROM BASE WHERE rank=1;
'''))